In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.interpolate import griddata
from sklearn.preprocessing import MinMaxScaler

### 1년치 데이터 불러오기

In [33]:
date_daily2 = pd.read_csv(r'미세먼지_daily_동_v2.csv', index_col=0)
a = pd.read_csv('노원구_list.csv',index_col = 0)

###위도 경도 범위를 0~1로 스케일링
aa0 = a['위도'].to_list()
aa1 = a['경도'].to_list()
result = list(map(lambda x, y: [x, y], aa0, aa1))
scaler = MinMaxScaler()
scaler.fit(result)
MinMaxScaler(copy=True, feature_range=(0, 1))
scaler.transform(result)
a = a.join(pd.DataFrame(scaler.transform(result)))
a.columns = ['경도','동','위도','위도_sc','경도_sc']
a = a[['동','위도','위도_sc','경도','경도_sc']]
temp = a.merge(date_daily2, on='동', how = 'outer')

In [34]:
###저장할 데이터 프레임 지정
pm10_dt1 = date_daily2[['동','pm10','date_daily']]

In [35]:
for idx in date_daily2['date_daily'].unique():
    ####빈 날짜 가져오기
    daylist = temp[temp['date_daily']==idx]
    alist = list(set(a['동'].tolist()).difference(set(daylist['동'].tolist())))
    alist2 = a[a['동'].isin(alist)]

    ###꼭지점을 평균값으로 지정
    Vertex = [{'위도_sc':0,
              '경도_sc':0,
              'pm10':daylist['pm10'].mean()
              },
             {'위도_sc':0,
              '경도_sc':1,
              'pm10':daylist['pm10'].mean()},
             {'위도_sc':1,
              '경도_sc':0,
              'pm10':daylist['pm10'].mean()},
             {'위도_sc':1,
              '경도_sc':1,
              'pm10':daylist['pm10'].mean()}]
    Vertex = pd.DataFrame(Vertex)

    ###미세먼지 데이터 보간
    x = daylist['위도_sc'].append(Vertex['위도_sc']).values
    y = daylist['경도_sc'].append(Vertex['경도_sc']).values
    z = daylist['pm10'].append(Vertex['pm10']).values
    xi = alist2['위도_sc']
    yi = alist2['경도_sc']
    gotpm10=griddata((x,y),z,(xi,yi),method='cubic')
    
    ###데이터 프레임으로 만들어서 추가하기
    temp_v1 = list(map(lambda x,x1: [x,x1, idx], alist2['동'], gotpm10))
    temp_v2 = pd.DataFrame(temp_v1)
    temp_v2.columns = ['동','pm10','date_daily']
    pm10_dt1 = pm10_dt1.append(temp_v2)

In [36]:
pm10_dt1 = pm10_dt1.sort_values('동').reset_index(drop=True)

### pm25

In [37]:
###저장할 데이터 프레임 지정
pm25_dt1 = date_daily2[['동','pm25','date_daily']]

In [38]:
for idx in date_daily2['date_daily'].unique():
    ####빈 날짜 가져오기
    daylist = temp[temp['date_daily']==idx]
    alist = list(set(a['동'].tolist()).difference(set(daylist['동'].tolist())))
    alist2 = a[a['동'].isin(alist)]

    ###꼭지점을 평균값으로 지정
    Vertex = [{'위도_sc':0,
              '경도_sc':0,
              'pm25':daylist['pm25'].mean()
              },
             {'위도_sc':0,
              '경도_sc':1,
              'pm25':daylist['pm25'].mean()},
             {'위도_sc':1,
              '경도_sc':0,
              'pm25':daylist['pm25'].mean()},
             {'위도_sc':1,
              '경도_sc':1,
              'pm25':daylist['pm25'].mean()}]
    Vertex = pd.DataFrame(Vertex)

    ###미세먼지 데이터 보간
    x = daylist['위도_sc'].append(Vertex['위도_sc']).values
    y = daylist['경도_sc'].append(Vertex['경도_sc']).values
    z = daylist['pm25'].append(Vertex['pm25']).values
    xi = alist2['위도_sc']
    yi = alist2['경도_sc']
    gotpm25=griddata((x,y),z,(xi,yi),method='cubic')
    
    ###데이터 프레임으로 만들어서 추가하기
    temp_v1 = list(map(lambda x,x1: [x,x1, idx], alist2['동'], gotpm25))
    temp_v2 = pd.DataFrame(temp_v1)
    temp_v2.columns = ['동','pm25','date_daily']
    pm25_dt1 = pm25_dt1.append(temp_v2)

In [39]:
pm25_dt1 = pm25_dt1.sort_values('date_daily').reset_index(drop=True)
pm25_dt1.merge(pm10_dt1,how = 'outer',on=['동','date_daily']).to_csv('노원구_pm보간데이터.csv')

In [40]:
pm25_dt1.merge(pm10_dt1,how = 'outer',on=['동','date_daily'])['동'].value_counts()

상계9동      365
월계2동      365
월계3동      365
하계2동      365
공릉2동      365
상계5동      365
상계8동      365
중계2,3동    365
중계1동      365
상계1동      365
월계1동      365
상계10동     365
공릉1동      365
상계2동      365
하계1동      365
상계3,4동    365
상계6,7동    365
중계4동      365
중계본동      365
Name: 동, dtype: int64

# 종로구 보간하기

In [22]:
date_daily2 = pd.read_csv(r'미세먼지_종로_daily_동_v2.csv', index_col=0)
date_daily2 = date_daily2[date_daily2['동']!='명동']
date_daily2 = date_daily2[date_daily2['동']!='천연동']
a = pd.read_csv('종로구_list.csv',index_col = 0)

###위도 경도 범위를 0~1로 스케일링
aa0 = a['위도'].to_list()
aa1 = a['경도'].to_list()
result = list(map(lambda x, y: [x, y], aa0, aa1))
scaler = MinMaxScaler()
scaler.fit(result)
MinMaxScaler(copy=True, feature_range=(0, 1))
scaler.transform(result)
a = a.join(pd.DataFrame(scaler.transform(result)))
a.columns = ['경도','동','위도','위도_sc','경도_sc']
a = a[['동','위도','위도_sc','경도','경도_sc']]
temp = a.merge(date_daily2, on='동', how = 'outer')

In [23]:
###저장할 데이터 프레임 지정
pm10_dt1 = date_daily2[['동','pm10','date_daily']]

In [24]:
for idx in date_daily2['date_daily'].unique():
    ####빈 날짜 가져오기
    daylist = temp[temp['date_daily']==idx]
    alist = list(set(a['동'].tolist()).difference(set(daylist['동'].tolist())))
    alist2 = a[a['동'].isin(alist)]

    ###꼭지점을 평균값으로 지정
    Vertex = [{'위도_sc':0,
              '경도_sc':0,
              'pm10':daylist['pm10'].mean()
              },
             {'위도_sc':0,
              '경도_sc':1,
              'pm10':daylist['pm10'].mean()},
             {'위도_sc':1,
              '경도_sc':0,
              'pm10':daylist['pm10'].mean()},
             {'위도_sc':1,
              '경도_sc':1,
              'pm10':daylist['pm10'].mean()}]
    Vertex = pd.DataFrame(Vertex)

    ###미세먼지 데이터 보간
    x = daylist['위도_sc'].append(Vertex['위도_sc']).values
    y = daylist['경도_sc'].append(Vertex['경도_sc']).values
    z = daylist['pm10'].append(Vertex['pm10']).values
    xi = alist2['위도_sc']
    yi = alist2['경도_sc']
    gotpm10=griddata((x,y),z,(xi,yi),method='cubic')
    
    ###데이터 프레임으로 만들어서 추가하기
    temp_v1 = list(map(lambda x,x1: [x,x1, idx], alist2['동'], gotpm10))
    temp_v2 = pd.DataFrame(temp_v1)
    temp_v2.columns = ['동','pm10','date_daily']
    pm10_dt1 = pm10_dt1.append(temp_v2)

In [25]:
pm10_dt1 = pm10_dt1.sort_values('동').reset_index(drop=True)
pm10_dt1.to_csv('종로구_pm10_보간데이터.csv')

### pm25

In [26]:
###저장할 데이터 프레임 지정
pm25_dt1 = date_daily2[['동','pm25','date_daily']]

In [27]:
for idx in date_daily2['date_daily'].unique():
    ####빈 날짜 가져오기
    daylist = temp[temp['date_daily']==idx]
    alist = list(set(a['동'].tolist()).difference(set(daylist['동'].tolist())))
    alist2 = a[a['동'].isin(alist)]

    ###꼭지점을 평균값으로 지정
    Vertex = [{'위도_sc':0,
              '경도_sc':0,
              'pm25':daylist['pm25'].mean()
              },
             {'위도_sc':0,
              '경도_sc':1,
              'pm25':daylist['pm25'].mean()},
             {'위도_sc':1,
              '경도_sc':0,
              'pm25':daylist['pm25'].mean()},
             {'위도_sc':1,
              '경도_sc':1,
              'pm25':daylist['pm25'].mean()}]
    Vertex = pd.DataFrame(Vertex)

    ###미세먼지 데이터 보간
    x = daylist['위도_sc'].append(Vertex['위도_sc']).values
    y = daylist['경도_sc'].append(Vertex['경도_sc']).values
    z = daylist['pm25'].append(Vertex['pm25']).values
    xi = alist2['위도_sc']
    yi = alist2['경도_sc']
    gotpm25=griddata((x,y),z,(xi,yi),method='cubic')
    
    ###데이터 프레임으로 만들어서 추가하기
    temp_v1 = list(map(lambda x,x1: [x,x1, idx], alist2['동'], gotpm25))
    temp_v2 = pd.DataFrame(temp_v1)
    temp_v2.columns = ['동','pm25','date_daily']
    pm25_dt1 = pm25_dt1.append(temp_v2)

In [28]:
pm25_dt1 = pm25_dt1.sort_values('date_daily').reset_index(drop=True)
pm25_dt1.to_csv('종로구_pm25_보간데이터.csv')

In [29]:
pm25_dt1.merge(pm10_dt1,how = 'outer',on=['동','date_daily']).to_csv('종로구_pm보간데이터.csv')

In [31]:
pm25_dt1.merge(pm10_dt1,how = 'outer',on=['동','date_daily'])['동'].value_counts()

종로 1,2,3,4가동    365
창신1동            365
창신2동            365
숭인2동            365
청운효자동           365
가회동             365
무악동             365
혜화동             365
부암동             365
이화동             365
평창동             365
교남동             365
삼청동             365
숭인1동            365
창신3동            365
사직동             365
종로5,6가동         365
Name: 동, dtype: int64